<a href="https://colab.research.google.com/github/edufsi/Projeto-Desafio/blob/Eduardo/Limpeza_Dados_Exportacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np

In [17]:
#Importar dados:
url_commodities = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/UNdata_Export_20240412_201846138.csv"
url_parceiros = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_330_202310_Major%20Trading%20Partners.csv"
url_exportacoes = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/SYB66_123_202310_Total%20Imports%20Exports%20and%20Balance%20of%20Trade.csv"
url_cambio = "https://raw.githubusercontent.com/edufsi/Projeto-Desafio/main/Dados/USD_BRL_Historical_Data.csv"


dados_commodities = pd.read_csv(url_commodities)
dados_parceiros = pd.read_csv(url_parceiros, encoding = "ISO-8859-1")
dados_exportacoes = pd.read_csv(url_exportacoes, encoding = "ISO-8859-1")
dados_cambio = pd.read_csv(url_cambio)

# LIMPEZA DOS DADOS DE EXPORTAÇÃO

In [18]:
# VISÃO GERAL
print(dados_exportacoes.head())

                   T21 Total imports, exports and balance of trade Unnamed: 2  \
0  Region/Country/Area                                         NaN       Year   
1                    1               Total, all countries or areas       1995   
2                    1               Total, all countries or areas       2005   
3                    1               Total, all countries or areas       2010   
4                    1               Total, all countries or areas       2015   

                             Unnamed: 3       Unnamed: 4  \
0                                Series  System of trade   
1  Imports CIF (millions of US dollars)              NaN   
2  Imports CIF (millions of US dollars)              NaN   
3  Imports CIF (millions of US dollars)              NaN   
4  Imports CIF (millions of US dollars)              NaN   

                 Unnamed: 5  Unnamed: 6 Unnamed: 7  \
0  System of trade footnote       Value  Footnotes   
1                       NaN   5,099,057     

In [19]:
print(dados_exportacoes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5054 entries, 0 to 5053
Data columns (total 9 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   T21                                          5054 non-null   object
 1   Total imports, exports and balance of trade  5053 non-null   object
 2   Unnamed: 2                                   5054 non-null   object
 3   Unnamed: 3                                   5054 non-null   object
 4   Unnamed: 4                                   4382 non-null   object
 5   Unnamed: 5                                   1 non-null      object
 6   Unnamed: 6                                   5054 non-null   object
 7   Unnamed: 7                                   1524 non-null   object
 8   Unnamed: 8                                   5054 non-null   object
dtypes: object(9)
memory usage: 355.5+ KB
None


In [20]:
# Os titulos das colunas estao errados. Eles estao na segunda coluna. Na primeira temos um titulo
# Podemos usar iloc para selecionar linhas. Passando um intervalo, uma lista ou um numero retornamos as linhas correspondentes


# Vamos tomar a primeira linha e defini-la como nossas colunas
dados_exportacoes.columns = dados_exportacoes.iloc[0]


# Agora que os titulos ja estao corretos, podemos excluir essa primeira linha que continha os títulos
dados_exportacoes.drop(0, inplace=True)


In [21]:
# Ainda há uma coluna "errada" no entanto. No arquivo original, foram associados valores para representar as regioes/paises/areas e há uma coluna sem titulo para a descricao
# Seria melhor aqui ter apenas uma coluna com os nomes das regiões, áreas e dos países. Temos que alter entao o nome da segunda coluna, essa sem titulo


lista = [x for x in dados_exportacoes.columns]
lista[1] = "Region/Country" # Substituindo o nome da coluna com nan por "Region/Country" (segunda coluna)

dados_exportacoes.columns = lista

print(dados_exportacoes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5053 entries, 1 to 5053
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Region/Country/Area       5053 non-null   object
 1   Region/Country            5053 non-null   object
 2   Year                      5053 non-null   object
 3   Series                    5053 non-null   object
 4   System of trade           4381 non-null   object
 5   System of trade footnote  0 non-null      object
 6   Value                     5053 non-null   object
 7   Footnotes                 1523 non-null   object
 8   Source                    5053 non-null   object
dtypes: object(9)
memory usage: 355.4+ KB
None


In [22]:
# Agora podemos retirar as colunas inúteis
# Podemos retirar a coluna "System of trade footnote", pois ela nao tem nenhum valor nao nulo
# Tambem podemos tirar a coluna de codificacao das regioes, ja que iremos nos referir a elas pelo nome
# A coluna Source também pode ser retirada pois apresenta apenas um valor

dados_exportacoes.drop(["System of trade footnote", "Region/Country/Area", "Source"], axis=1, inplace=True)

In [ ]:
# A coluna de valores esta como tipo objeto. Precisamos tranformá-la em numero. Nao temos numeros quebrados, podemos transformá-la em numeros inteiros
# Precisamos retirar as virgulas dos numeros, para entao transforma-los em inteiros. Vamos transformar em inteiros de 8 bytes para ter certeza de não ter problemas de overflow

dados_exportacoes['Value'] = dados_exportacoes['Value'].apply(lambda x: x.replace(',', '')).astype(np.int64)

In [ ]:
# Finalmente temos um dataframe em que todas as colunas dizem algo util


# Vamos agora tirar as regiões e uniões para o lado, deixando só os países

regioes = ['Total, all countries or areas', 'Africa', 'Northern Africa',
 'Sub-Saharan Africa', 'Eastern Africa', 'Middle Africa', 'Southern Africa',
 'Western Africa', 'Americas', 'Northern America',
 'Latin America & the Caribbean', 'Caribbean', 'Central America',
 'South America', 'Asia', 'Central Asia', 'Eastern Asia', 'South-eastern Asia',
 'Southern Asia', 'Western Asia', 'Europe', 'Eastern Europe',
 'Northern Europe', 'Southern Europe', 'Western Europe', 'Oceania',
 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia']



exportacoes_regioes = dados_exportacoes.query('`Region/Country` in @regioes')
exportacoes_paises = dados_exportacoes.query('`Region/Country` not in @regioes')

In [ ]:
# Terminamos com 2 dataframe: um para as regiões e agrupamentos e outro com cada país individual